In [6]:
import os
import sys

notebook_dir = os.getcwd()
project_root_path = os.path.dirname(notebook_dir)
sys.path.insert(0, project_root_path)

from src.preprocessing.CUB import preprocessing_CUB
from src.utils import *
from src.config import PROJECT_ROOT
from src.training import run_epoch_x_to_c

from src.utils import find_class_imbalance
from src.config import CUB_CONFIG
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [7]:
N_TRIMMED_CONCEPTS = CUB_CONFIG['N_TRIMMED_CONCEPTS']

In [8]:
torch.mps.empty_cache()

In [9]:
torch.manual_seed(42)
concept_labels, train_loader, test_loader = preprocessing_CUB(training=False, class_concepts=True, verbose=True)

Found 11788 images.
Processing in 185 batches of size 64 (for progress reporting)...


Processing batches: 100%|█████████████████████| 185/185 [01:01<00:00,  3.03it/s]



Finished processing.
Successfully transformed: 11788 images.
Found 11788 unique images.
Found 312 unique concepts.
Generated concept matrix with shape: (11788, 312)
Found 200 classes.
Found labels for 11788 images.
Generated one-hot matrix with shape: (11788, 200)
Split complete: 5994 train images, 5794 test images.
Dataset initialized with 5994 pre-sorted items.
Dataset initialized with 5794 pre-sorted items.


**Find device to run model on (CPU or GPU).**

In [10]:
device = torch.device("cuda" if torch.cuda.is_available()
                    else "mps" if torch.backends.mps.is_available()
                    else "cpu")
print(f"Using device: {device}")

Using device: mps


### Loss


In [11]:
use_weighted_loss = True # Set to False for simple unweighted loss

if use_weighted_loss:
    concept_weights = find_class_imbalance(concept_labels)
    attr_criterion = [nn.BCEWithLogitsLoss(weight=torch.tensor([ratio], device=device, dtype=torch.float))
                    for ratio in concept_weights]
else:
    attr_criterion = [nn.BCEWithLogitsLoss() for _ in range(N_TRIMMED_CONCEPTS)]

In [12]:
def get_outputs_as_array(outputs, n_concepts):
    # Initialize an empty list to collect batches
    batch_results = []

    for i in range(len(outputs)):
        batch_size = outputs[i].shape[0]

        # Create a batch matrix with N_CONCEPTS number of columns
        batch_matrix = np.zeros((batch_size, n_concepts))

        for instance_idx in range(batch_size):
            # Extract, convert, and flatten data for the current concept
            instance_data = outputs[i][instance_idx].detach().cpu().numpy().flatten()
            batch_matrix[instance_idx, :] = instance_data

        # Add this consistently shaped batch matrix to our collection
        batch_results.append(batch_matrix)

    return np.vstack(batch_results)

# Load instance-based model

In [13]:
# model = torch.load(os.path.join(PROJECT_ROOT, 'models', 'CUB', 'instance_level_model.pth'), map_location=device, weights_only=False)
# # model = torch.load(os.path.join(PROJECT_ROOT, 'notebook', 'x_to_c_best_model.pth'), map_location=device, weights_only=False)

In [14]:
def get_outputs(loader, split_name):
    if loader:
        with torch.no_grad():
            shuffled_concept_labels = []
            shuffled_img_labels = []

            # Iterate through all batches
            for batch in loader:
                _, concept_labels, image_labels, _ = batch
                # Append batch labels to our list
                shuffled_concept_labels.append(concept_labels)
                shuffled_img_labels.append(image_labels)

            # Concatenate all batches into a single tensor
            shuffled_concept_labels = torch.cat(shuffled_concept_labels, dim=0)
            shuffled_img_labels = torch.cat(shuffled_img_labels, dim=0)

            test_loss, test_acc, outputs = run_epoch_x_to_c(
                model, loader, attr_criterion, optimizer=None, n_concepts=N_TRIMMED_CONCEPTS, device=device,
                return_outputs='sigmoid', verbose=True
            )

    # print(f"Shuffled labels shape: {shuffled_img_labels.shape}")
    output_dir = os.path.join(PROJECT_ROOT, 'output', 'CUB')
    os.makedirs(output_dir, exist_ok=True)
    np.save(os.path.join(output_dir, f'C_{split_name}_instance.npy'), shuffled_concept_labels)
    np.save(os.path.join(output_dir, f'Y_{split_name}_instance.npy'), shuffled_img_labels)
    print(f'Best Model Summary   | Loss: {test_loss:.4f} | Acc: {test_acc:.3f}')

    output_array = get_outputs_as_array(outputs, N_TRIMMED_CONCEPTS)
    print(f"Final shape: {output_array.shape}")

    np.save(os.path.join(output_dir, f'C_hat_sigmoid_{split_name}_instance.npy'), output_array)

In [15]:
# get_outputs(train_loader, 'train')

# LOAD BEST MODEL FROM CBM PAPER

In [16]:
best_model = os.path.join(PROJECT_ROOT, 'models', 'CUB', 'best_model_1.pth')
model = torch.load(best_model, map_location=device, weights_only=False)
print("Best model loaded.")

/Users/pb/.pyenv/versions/3.11.9/lib/python3.11/site-packages/torch/serialization.py:1580: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv2d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/Users/pb/.pyenv/versions/3.11.9/lib/python3.11/site-packages/torch/serialization.py:1580: SourceChangeWarning: source code of class 'torch.nn.modules.batchnorm.BatchNorm2d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/Users/pb/.pyenv/versions/3.11.9/lib/python3.11/site-packages/torch/serialization.py:1580: SourceChangeWarning: source code of class 'torch.nn.modules.container.ModuleList' has changed. you can retrie

Best model loaded.


In [17]:
get_outputs(train_loader, 'train')

Best Model Summary   | Loss: 11.4714 | Acc: 97.410
Final shape: (5994, 112)


In [18]:
get_outputs(test_loader, 'test')

Best Model Summary   | Loss: 11.6737 | Acc: 94.864
Final shape: (5794, 112)


In [19]:
# best_model = os.path.join(PROJECT_ROOT, 'models', 'CUB', 'best_model_2.pth')
# get_outputs(train_loader, 'train')

In [20]:
# best_model = os.path.join(PROJECT_ROOT, 'models', 'CUB', 'best_model_3.pth')
# get_outputs(train_loader, 'train')
